In [ ]:
import os
from pycparser import c_parser, c_ast

class CodeComparator:
    def __init__(self):
        self.compare_data_structures = False
        self.compare_control_structures = False

    def enable_priority(self, priority):
        """
        Enable comparison for the given priority ('data' or 'control').
        """
        if priority == 'data':
            self.compare_data_structures = True
        elif priority == 'control':
            self.compare_control_structures = True
        else:
            self.compare_control_structures = True
            self.compare_data_structures = True

    def preprocess_file(self, file_path):
        """
        Preprocess a C source file to handle macros and includes.
        Returns the preprocessed code as a string.
        """
        try:

            import subprocess
            preprocessed_code = subprocess.check_output(
                ["gcc", "-E", file_path], universal_newlines=True
            )
            return preprocessed_code
        except FileNotFoundError:
            print("Error: gcc is not installed or not in PATH.")
            exit()
        except Exception as e:
            print(f"Error during preprocessing: {e}")
            exit()

    def extract_data_structures(self, ast_node):
        """
        Recursively extract data structure declarations (e.g., arrays, structs) from the AST.
        """
        data_structures = []
        if ast_node is None:
            return data_structures

        if isinstance(ast_node, c_ast.Struct):
            data_structures.append(f"struct {ast_node.name}")
        elif isinstance(ast_node, c_ast.ArrayDecl):
            data_structures.append("array")

        if hasattr(ast_node, 'children'):
            for _, child in ast_node.children():
                data_structures.extend(self.extract_data_structures(child))

        return data_structures

    def extract_control_structures(self, ast_node):
        """
        Recursively extract control structure declarations (e.g., for, while, if) from the AST.
        """
        control_structures = []
        if ast_node is None:
            return control_structures

        if isinstance(ast_node, c_ast.For):
            control_structures.append("for loop")
        elif isinstance(ast_node, c_ast.While):
            control_structures.append("while loop")
        elif isinstance(ast_node, c_ast.DoWhile):
            control_structures.append("do-while loop")
        elif isinstance(ast_node, c_ast.If):
            control_structures.append("if statement")
        elif isinstance(ast_node, c_ast.Switch):
            control_structures.append("switch statement")

        if hasattr(ast_node, 'children'):
            for _, child in ast_node.children():
                control_structures.extend(self.extract_control_structures(child))

        return control_structures

    def compare_files(self, file1, file2):
        """
        Compare data structures and control structures used in two C source files
        based on enabled priorities.
        """
        parser = c_parser.CParser()

        try:

            code1 = self.preprocess_file(file1)
            code2 = self.preprocess_file(file2)

            ast1 = parser.parse(code1)
            ast2 = parser.parse(code2)

            result = {}


            if self.compare_data_structures==True:
                data_structures1 = set(self.extract_data_structures(ast1))
                data_structures2 = set(self.extract_data_structures(ast2))

                result['data_structures'] = {
                    "common": data_structures1 & data_structures2,
                    "unique_to_file1": data_structures1 - data_structures2,
                    "unique_to_file2": data_structures2 - data_structures1
                }


            if self.compare_control_structures==True:
                control_structures1 = set(self.extract_control_structures(ast1))
                control_structures2 = set(self.extract_control_structures(ast2))

                result['control_structures'] = {
                    "common": control_structures1 & control_structures2,
                    "unique_to_file1": control_structures1 - control_structures2,
                    "unique_to_file2": control_structures2 - control_structures1
                }

            return result

        except Exception as e:
            print(f"Error parsing files: {e}")
            return None


if __name__ == "__main__":

    file1 = "/content/arth.c"
    file2 = "/content/arth1.c"

    comparator = CodeComparator()
    value = input("Prompt message [1:data 2: control 3: Both]: ")

    if value=="1":
      comparator.enable_priority('data')
    elif value=="2":
      comparator.enable_priority('control')
    else:
      comparator.enable_priority('both')

    if not (os.path.exists(file1) and os.path.exists(file2)):
        print("Both source files must exist.")
        exit()

    result = comparator.compare_files(file1, file2)
    if result:
        print("\nComparison Result:")
        if 'data_structures' in result:
            print(f"Student file lacks {result['data_structures']['unique_to_file1']} data structure")

        if 'control_structures' in result:
            print(f"Student file lacks {result['control_structures']['unique_to_file1']} Control structure")
